In [41]:
import os
import csv
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import interact
import time
import dask.dataframe as dd
import numpy as np
import folium
import matplotlib.pyplot as plt
import seaborn as sns

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

import


In [3]:
# Function to get the data from the csv files and return a dataframe
def csv_to_df(filepath):
    try:
        return pd.read_csv(filepath, sep=",")
    except pd.errors.ParserError:
        return pd.read_csv(filepath, sep=";")


passagierfrequenz_df = csv_to_df("raw_data/csv-4.csv")
haltestelle_df = csv_to_df("raw_data/haltestelle.csv")
haltepunkt_df = csv_to_df("raw_data/haltepunkt.csv")

In [4]:
fahrzeiten_filenames = [
    filename
    for filename in os.listdir("raw_data/fahrzeiten/")
    if filename.endswith(".csv")
]
fahrzeiten_head = pd.read_csv(
    "raw_data/fahrzeiten/" + fahrzeiten_filenames[0], nrows=0
).columns


def get_fahrzeiten_dask_df(filename, columns=None):
    if columns is None:
        columns = fahrzeiten_head
    df = dd.read_csv("raw_data/fahrzeiten/" + filename, usecols=columns)
    return df.compute()


fahrzeiten_dfs = {
    filename: get_fahrzeiten_dask_df(filename) for filename in fahrzeiten_filenames
}

In [ ]:
# Create widgets
output = widgets.Output()
dropdown = widgets.Dropdown(
    options=fahrzeiten_filenames, layout=widgets.Layout(width="auto")
)
next_button = widgets.Button(description="Next")
prev_button = widgets.Button(description="Previous")
head_button = widgets.Button(description="Head")
tail_button = widgets.Button(description="Tail")
output = widgets.Output(layout=widgets.Layout(overflow_x="scroll"))

# Initialize index
index = [0]


# Event handlers for button clicks
def on_next_button_clicked(b):
    index[0] = (index[0] + 1) % len(fahrzeiten_filenames)
    dropdown.value = fahrzeiten_filenames[index[0]]


def on_prev_button_clicked(b):
    index[0] = (index[0] - 1) % len(fahrzeiten_filenames)
    dropdown.value = fahrzeiten_filenames[index[0]]


def on_head_button_clicked(b):
    with output:
        output.clear_output()
        display(fahrzeiten_dfs[dropdown.value].head())


def on_tail_button_clicked(b):
    with output:
        output.clear_output()
        display(fahrzeiten_dfs[dropdown.value].tail())


# Attach event handlers to buttons
next_button.on_click(on_next_button_clicked)
prev_button.on_click(on_prev_button_clicked)
head_button.on_click(on_head_button_clicked)
tail_button.on_click(on_tail_button_clicked)

title = widgets.HTML('<h2 style="text-align: center;">Fahrzeiten: SOLL und IST</h2>')
box_layout = widgets.Layout(display="flex", justify_content="center")
display(
    widgets.VBox(
        [
            title,
            widgets.HBox([dropdown], layout=box_layout),
            widgets.HBox(
                [head_button, prev_button, next_button, tail_button], layout=box_layout
            ),
            widgets.HBox([output], layout=box_layout),
        ],
        layout=box_layout,
    )
)

In [ ]:
# Create a dictionary of your DataFrames
dfs = {
    "Haltepunkte_df": haltepunkt_df,
    "Haltestelle_df": haltestelle_df,
    "Passagierfrequenz_df": passagierfrequenz_df,
}
# Create widgets
output = widgets.Output()
dropdown = widgets.Dropdown(options=list(dfs.keys()))
head_button = widgets.Button(description="Head")
tail_button = widgets.Button(description="Tail")
describe_button = widgets.Button(description="Describe")
shape_button = widgets.Button(description="Shape")
isnull_button = widgets.Button(description="IsNull")
dtype_button = widgets.Button(description="Dtype")
output = widgets.Output()


# Event handlers for button clicks
def on_head_button_clicked(b):
    with output:
        output.clear_output()
        display(dfs[dropdown.value].head())


def on_tail_button_clicked(b):
    with output:
        output.clear_output()
        display(dfs[dropdown.value].tail())


def on_describe_button_clicked(b):
    with output:
        output.clear_output()
        display(dfs[dropdown.value].describe())


def on_shape_button_clicked(b):
    with output:
        output.clear_output()
        print(dfs[dropdown.value].shape)


def on_isnull_button_clicked(b):
    with output:
        output.clear_output()
        display(dfs[dropdown.value].isnull().sum())


def on_info_button_clicked(b):
    with output:
        output.clear_output()
        display(dfs[dropdown.value].info())


def on_dtype_button_clicked(b):
    with output:
        output.clear_output()
        display(dfs[dropdown.value].dtypes)


# Attach event handlers to buttons
head_button.on_click(on_head_button_clicked)
tail_button.on_click(on_tail_button_clicked)
describe_button.on_click(on_describe_button_clicked)
shape_button.on_click(on_shape_button_clicked)
isnull_button.on_click(on_isnull_button_clicked)


title = widgets.HTML('<h2 style="text-align: center;">Other CSVs</h2>')
box_layout = widgets.Layout(display="flex", justify_content="center")
display(
    widgets.VBox(
        [
            title,
            widgets.HBox([dropdown], layout=box_layout),
            widgets.HBox(
                [
                    head_button,
                    tail_button,
                    describe_button,
                    shape_button,
                    isnull_button,
                    dtype_button,
                ],
                layout=box_layout,
            ),
            widgets.HBox([output], layout=box_layout),
        ],
        layout=box_layout,
    )
)

In [26]:
display(haltestelle_df.dtypes)
display(haltepunkt_df.dtypes)
display(passagierfrequenz_df.dtypes)

halt_id            int64
halt_diva          int64
halt_kurz         object
halt_lang         object
halt_ist_aktiv      bool
dtype: object

halt_punkt_id             int64
halt_punkt_diva           int64
halt_id                   int64
GPS_Latitude            float64
GPS_Longitude           float64
GPS_Bearing             float64
halt_punkt_ist_aktiv       bool
dtype: object

code_codice               object
uic                      float64
bahnhof_gare_stazione     object
kt_ct_cantone             object
isb_gi                    object
jahr_annee_anno          float64
dtv_tjm_tgm              float64
dwv_tmjo_tfm             float64
dnwv_tmjno_tmgnl         float64
evu_ef_itf                object
bemerkungen               object
remarques                 object
note                      object
remarks                   object
geopos                    object
lod                       object
dtype: object

In [30]:
bemerkungen_df = passagierfrequenz_df["bemerkungen"].dropna()
remarks_df = passagierfrequenz_df["remarques"].dropna()
note_df = passagierfrequenz_df["note"].dropna()

display(bemerkungen_df)
display(remarks_df)
display(note_df)

2                                               Ohne TPC.
3                                               Ohne TPC.
6           Weniger als 50 Ein- und Aussteigende pro Tag.
18      Weniger als 50 Ein- und Aussteigende pro Nicht...
19          Weniger als 50 Ein- und Aussteigende pro Tag.
                              ...                        
2289                           Inklusive Bahnverkehr MVR.
2294        Weniger als 50 Ein- und Aussteigende pro Tag.
2296        Weniger als 50 Ein- und Aussteigende pro Tag.
2302                                            Ohne AVA.
2304    Durchschnittswert 2018 durch Streckensperrung ...
Name: bemerkungen, Length: 387, dtype: object

2                                               Sans TPC.
3                                               Sans TPC.
6                         Moins de 50 personnes par jour.
18              Moins de 50 personnes par jour non ouvré.
19                        Moins de 50 personnes par jour.
                              ...                        
2289                                   Trains MVR inclus.
2294                      Moins de 50 personnes par jour.
2296                      Moins de 50 personnes par jour.
2302                                            Sans AVA.
2304    Valeur moyenne 2018 à la baisse en raison de l...
Name: remarques, Length: 387, dtype: object

2                                              Senza TPC.
3                                              Senza TPC.
6                           Meno di 50 persone al giorno.
18           Meno di 50 persone al giorno non lavorativo.
19                          Meno di 50 persone al giorno.
                              ...                        
2289                                     Incl. treni MVR.
2294                        Meno di 50 persone al giorno.
2296                        Meno di 50 persone al giorno.
2302                                           Senza AVA.
2304    Valore medio 2018 compromesso dallo sbarrament...
Name: note, Length: 387, dtype: object

In [33]:
# Create widgets
output2 = widgets.Output()
dropdown2 = widgets.Dropdown(options=fahrzeiten_filenames)
describe_button = widgets.Button(description="Describe")
shape_button = widgets.Button(description="Shape")
isnull_button = widgets.Button(description="Is Null")
dtype_button = widgets.Button(description="Dtype")


# Event handlers for button clicks
def on_describe_button_clicked(b):
    with output2:
        output2.clear_output()
        display(fahrzeiten_dfs[dropdown2.value].describe())


def on_shape_button_clicked(b):
    with output2:
        output2.clear_output()
        print(fahrzeiten_dfs[dropdown2.value].shape)


def on_isnull_button_clicked(b):
    with output2:
        output2.clear_output()
        display(fahrzeiten_dfs[dropdown2.value].isnull().sum())


def on_dtype_button_clicked(b):
    with output2:
        output2.clear_output()
        if dropdown2.value in fahrzeiten_dfs:
            if not fahrzeiten_dfs[dropdown2.value].empty:
                display(fahrzeiten_dfs[dropdown2.value].dtypes)
            else:
                print("DataFrame is empty.")
        else:
            print("Key not found in dictionary.")


# Attach event handlers to buttons
describe_button.on_click(on_describe_button_clicked)
shape_button.on_click(on_shape_button_clicked)
isnull_button.on_click(on_isnull_button_clicked)

# Display widgets
title = widgets.HTML('<h2 style="text-align: center;">Fahrzeiten: SOLL und IST</h2>')
box_layout = widgets.Layout(display="flex", justify_content="center")
display(
    widgets.VBox(
        [
            title,
            widgets.HBox([dropdown2], layout=box_layout),
            widgets.HBox(
                [describe_button, shape_button, isnull_button, dtype_button],
                layout=box_layout,
            ),
            widgets.HBox([output2], layout=box_layout),
        ],
        layout=box_layout,
    )
)

Missing Data


In [ ]:
Unique_Haltepunkt = haltepunkt_df["Haltepunktbezeichnung"].unique()
Unique_haltestelle = haltestelle_df["Haltestellenbezeichnung"].unique()

In [34]:
display(fahrzeiten_dfs["Fahrzeiten_SOLL_IST_20220102_20220108.csv"].dtypes)

linie                    int64
richtung                 int64
betriebsdatum           object
fahrzeug                 int64
kurs                     int64
seq_von                  int64
halt_diva_von            int64
halt_punkt_diva_von      int64
halt_kurz_von1          object
datum_von               object
soll_an_von              int64
ist_an_von               int64
soll_ab_von              int64
ist_ab_von               int64
seq_nach                 int64
halt_diva_nach           int64
halt_punkt_diva_nach     int64
halt_kurz_nach1         object
datum_nach              object
soll_an_nach             int64
ist_an_nach1             int64
soll_ab_nach             int64
ist_ab_nach              int64
fahrt_id                 int64
fahrweg_id               int64
fw_no                    int64
fw_typ                   int64
fw_kurz                  int64
fw_lang                 object
umlauf_von               int64
halt_id_von              int64
halt_id_nach             int64
halt_pun

In [8]:
display(fahrzeiten_dfs["Fahrzeiten_SOLL_IST_20220102_20220108.csv"].columns)

Index(['linie', 'richtung', 'betriebsdatum', 'fahrzeug', 'kurs', 'seq_von',
       'halt_diva_von', 'halt_punkt_diva_von', 'halt_kurz_von1', 'datum_von',
       'soll_an_von', 'ist_an_von', 'soll_ab_von', 'ist_ab_von', 'seq_nach',
       'halt_diva_nach', 'halt_punkt_diva_nach', 'halt_kurz_nach1',
       'datum_nach', 'soll_an_nach', 'ist_an_nach1', 'soll_ab_nach',
       'ist_ab_nach', 'fahrt_id', 'fahrweg_id', 'fw_no', 'fw_typ', 'fw_kurz',
       'fw_lang', 'umlauf_von', 'halt_id_von', 'halt_id_nach',
       'halt_punkt_id_von', 'halt_punkt_id_nach'],
      dtype='object')

In [9]:
selected_columns_df = fahrzeiten_dfs["Fahrzeiten_SOLL_IST_20220102_20220108.csv"][
    ["linie", "halt_id_von", "halt_punkt_id_von", "halt_kurz_von1"]
]

display(selected_columns_df.columns)

Index(['linie', 'halt_id_von', 'halt_punkt_id_von', 'halt_kurz_von1'], dtype='object')

In [36]:
all_haltestellen_df = pd.merge(haltestelle_df, haltepunkt_df, on="halt_id", how="outer")

display(all_haltestellen_df)

,halt_id,halt_diva,halt_kurz,halt_lang,halt_ist_aktiv,halt_punkt_id,halt_punkt_diva,GPS_Latitude,GPS_Longitude,GPS_Bearing,halt_punkt_ist_aktiv
0,143,2570,BirWSL,"Birmensdorf ZH, Sternen/WSL",True,303,51,47.360017,8.456337,85.0,False
1,143,2570,BirWSL,"Birmensdorf ZH, Sternen/WSL",True,304,50,47.360153,8.456180,270.0,False
2,143,2570,BirWSL,"Birmensdorf ZH, Sternen/WSL",True,10982,50,47.360153,8.456180,266.0,False
3,143,2570,BirWSL,"Birmensdorf ZH, Sternen/WSL",True,13469,51,47.360035,8.456297,85.0,False
4,143,2570,BirWSL,"Birmensdorf ZH, Sternen/WSL",True,13485,50,47.360154,8.456141,266.0,False
...,...,...,...,...,...,...,...,...,...,...,...
19686,3338,7070,OBER,"Zürich, Obere Hornhalde",True,54979,51,47.331647,8.538054,328.0,True
19687,3339,7065,OGAS,"Oetwil an der Limmat, Gässliacker",True,54811,51,47.429765,8.392162,315.0,True
19688,3339,7065,OGAS,"Oetwil an der Limmat, Gässliacker",True,54820,50,47.429647,8.392346,135.0,True
19689,3360,6923,ZNIE,"Zollikon, Niederhofenrain",True,54815,51,47.347797,8.576690,270.0,True


In [10]:
# Convert pandas DataFrames to Dask DataFrames
selected_columns_dd = dd.from_pandas(selected_columns_df, npartitions=10)
haltestelle_dd = dd.from_pandas(haltestelle_df, npartitions=10)
haltepunkt_dd = dd.from_pandas(haltepunkt_df, npartitions=10)

# Perform the merge operation
merged_dd = selected_columns_dd.merge(
    haltestelle_dd, left_on="halt_id_von", right_on="halt_id"
)
final_dd = merged_dd.merge(haltepunkt_dd, left_on="halt_id_von", right_on="halt_id")

# Convert Dask DataFrame back to pandas DataFrame
final_df = final_dd.compute()

# Select only the necessary columns
bus_stops_for_each_line_df = final_df[
    ["linie", "halt_id_von", "halt_diva", "halt_kurz", "GPS_Latitude", "GPS_Longitude"]
]

In [17]:
display(bus_stops_for_each_line_df)

bus_stops_for_each_line_df = bus_stops_for_each_line_df.drop_duplicates(
    subset=["GPS_Latitude", "GPS_Longitude"]
)
bus_stops_for_each_line_df = bus_stops_for_each_line_df.dropna(
    subset=["GPS_Latitude", "GPS_Longitude"]
)

display(bus_stops_for_each_line_df)

,linie,halt_id_von,halt_diva,halt_kurz,GPS_Latitude,GPS_Longitude
0,32,2756,3029,HSBB,47.386308,8.517975
1,32,2756,3029,HSBB,NaN,NaN
2,32,2756,3029,HSBB,47.385184,8.516973
3,32,2756,3029,HSBB,47.386570,8.518749
4,32,2756,3029,HSBB,47.385343,8.517267
...,...,...,...,...,...,...
6639636,307,2030,6234,SCHN,47.401128,8.448775
6639637,307,2030,6234,SCHN,47.400991,8.448958
6639638,307,2030,6234,SCHN,47.401118,8.448868
6639639,307,2030,6234,SCHN,47.401009,8.448972


,linie,halt_id_von,halt_diva,halt_kurz,GPS_Latitude,GPS_Longitude
0,32,2756,3029,HSBB,47.386308,8.517975
2,32,2756,3029,HSBB,47.385184,8.516973
3,32,2756,3029,HSBB,47.386570,8.518749
4,32,2756,3029,HSBB,47.385343,8.517267
7,32,2756,3029,HSBB,47.386405,8.518163
...,...,...,...,...,...,...
6639636,307,2030,6234,SCHN,47.401128,8.448775
6639637,307,2030,6234,SCHN,47.400991,8.448958
6639638,307,2030,6234,SCHN,47.401118,8.448868
6639639,307,2030,6234,SCHN,47.401009,8.448972


In [12]:
# Erstellen Sie eine Karte mit einem Startpunkt (z. B. den geografischen Koordinaten einer Stadt).
m = folium.Map(location=[47, 8], zoom_start=10)

# Erhalten Sie die eindeutigen Werte in der "geopos" Spalte
unique_geopos = passagierfrequenz_df["geopos"].unique()

# Filter out the wrong values
unique_geopos = [geopos for geopos in unique_geopos if type(geopos) == str]

# Fügen Sie Marker für jede eindeutige geopos hinzu
# for geopos in unique_geopos:
#     # Teilen Sie den geopos String in latitude und longitude
#     latitude, longitude = geopos.split(',')

#     folium.Marker(
#         location=[latitude, longitude],
#         icon=folium.Icon(color="red")
#     ).add_to(m)


# haltepunkte_df is a dataframe with all rows from haltepunkt_df where GPS_Latitude and GPS_Longitude are not NaN
haltepunkte_df = haltepunkt_df[
    haltepunkt_df["GPS_Latitude"].notna() & haltepunkt_df["GPS_Longitude"].notna()
]

# Nur die Zeilen, wo eine Unique Komination von GPS_Latitude und GPS_Longitude existiert
haltepunkte_df = haltepunkte_df.drop_duplicates(
    subset=["GPS_Latitude", "GPS_Longitude"]
)

haltepunkte_df = haltepunkte_df.head(10)

locations = []

for index, row in haltepunkte_df.iterrows():
    location = [row["GPS_Latitude"], row["GPS_Longitude"]]
    locations.append(location)
    folium.Marker(location=location, icon=folium.Icon(color="blue")).add_to(m)

# Verbinden Sie die Haltepunkte mit Linien
folium.PolyLine(locations, color="red", weight=2.5, opacity=1).add_to(m)

m

# folium.TileLayer('Mapbox Bright').add_to(my_map)

Basic Analysis
